<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1fUStL32GseSXEQyu1TpgEs1Mm1mP8IUx?usp=sharing)

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Join Innovation Community

Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)

# Gemini 3 Flash Testing Notebook

This notebook demonstrates the capabilities of Google's **Gemini 3 Flash** model - a fast, efficient model optimized for speed while maintaining high quality outputs.

**What we'll cover:**
- Basic text generation
- Streaming responses
- Multimodal capabilities (image understanding)
- Structured outputs with Pydantic
- Function calling
- Google Search integration

**Prerequisites:**
- Google Cloud Project or AI Studio Account
- API Key from [Google AI Studio](https://aistudio.google.com/)

### 1. Installation
Install the Google GenAI SDK and required dependencies.

In [ ]:
%pip install -q google-genai pydantic pillow requests

### 2. Setup & Authentication
Configure your API key. It is recommended to use Google Colab's `userdata` for security.

In [ ]:
import os
from google import genai
from google.genai import types

try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except ImportError:
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY', 'YOUR_API_KEY_HERE')

os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

client = genai.Client(api_key=GOOGLE_API_KEY)

MODEL_ID = "gemini-2.5-flash-preview-05-20"
print(f"Using model: {MODEL_ID}")

### 3. Basic Text Generation
Test basic text generation capabilities with Gemini 3 Flash.

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Explain quantum computing in 3 sentences for a 10-year-old."
)

print(response.text)

### 4. Streaming Responses
Stream responses for real-time output - ideal for chat applications.

In [ ]:
response = client.models.generate_content_stream(
    model=MODEL_ID,
    contents="Write a short poem about artificial intelligence and creativity."
)

for chunk in response:
    print(chunk.text, end="", flush=True)

### 5. System Instructions
Guide the model's behavior with system instructions.

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="What's the best programming language?",
    config=types.GenerateContentConfig(
        system_instruction="You are a wise programming mentor who always gives balanced, nuanced advice. You never say one language is 'the best' - instead you explain trade-offs."
    )
)

print(response.text)

### 6. Multimodal - Image Understanding
Test Gemini 3 Flash's vision capabilities with image analysis.

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import base64

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a7/Camponotus_flavomarginatus_ant.jpg/800px-Camponotus_flavomarginatus_ant.jpg"
image_response = requests.get(image_url)
image = Image.open(BytesIO(image_response.content))

display(image)

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        "Describe this image in detail. What species is this? What interesting facts can you share?",
        image
    ]
)

print(response.text)

### 7. Structured Outputs with Pydantic
Get type-safe, structured JSON responses using Pydantic models.

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional

class CodeReview(BaseModel):
    code_quality: int = Field(description="Score from 1-10")
    issues: List[str] = Field(description="List of identified issues")
    suggestions: List[str] = Field(description="Improvement suggestions")
    security_concerns: Optional[List[str]] = Field(description="Any security issues found")

code_snippet = """
def get_user(user_id):
    query = f"SELECT * FROM users WHERE id = {user_id}"
    result = db.execute(query)
    return result
"""

response = client.models.generate_content(
    model=MODEL_ID,
    contents=f"Review this Python code and identify issues:\n{code_snippet}",
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=CodeReview,
    ),
)

review = CodeReview.model_validate_json(response.text)
print(f"Code Quality: {review.code_quality}/10")
print(f"\nIssues Found:")
for issue in review.issues:
    print(f"  - {issue}")
print(f"\nSecurity Concerns:")
for concern in review.security_concerns or []:
    print(f"  ⚠️ {concern}")

### 8. Function Calling
Define custom functions that the model can call to perform actions.

In [ ]:
def get_weather(location: str, unit: str = "celsius") -> dict:
    """Get the current weather for a location."""
    weather_data = {
        "new york": {"temp": 22, "condition": "Sunny"},
        "london": {"temp": 15, "condition": "Cloudy"},
        "tokyo": {"temp": 28, "condition": "Humid"},
    }
    data = weather_data.get(location.lower(), {"temp": 20, "condition": "Unknown"})
    if unit == "fahrenheit":
        data["temp"] = data["temp"] * 9/5 + 32
    return {"location": location, "temperature": data["temp"], "unit": unit, "condition": data["condition"]}

def calculate(expression: str) -> dict:
    """Evaluate a mathematical expression."""
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return {"expression": expression, "result": result}
    except Exception as e:
        return {"expression": expression, "error": str(e)}

tools = [get_weather, calculate]

response = client.models.generate_content(
    model=MODEL_ID,
    contents="What's the weather in Tokyo? Also, what's 15% of 340?",
    config=types.GenerateContentConfig(
        tools=tools,
    ),
)

print(response.text)

### 9. Google Search Integration
Ground responses with real-time information from Google Search.

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="What are the latest developments in AI agents as of this week?",
    config=types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
    ),
)

print(response.text)

### 10. Multi-turn Conversation
Maintain context across multiple turns in a conversation.

In [ ]:
chat = client.chats.create(
    model=MODEL_ID,
    config=types.GenerateContentConfig(
        system_instruction="You are a helpful AI tutor specializing in machine learning."
    )
)

response1 = chat.send_message("What is gradient descent?")
print("User: What is gradient descent?")
print(f"AI: {response1.text}\n")

response2 = chat.send_message("Can you give me a simple analogy?")
print("User: Can you give me a simple analogy?")
print(f"AI: {response2.text}\n")

response3 = chat.send_message("What are common variants of it?")
print("User: What are common variants of it?")
print(f"AI: {response3.text}")

### 11. Temperature & Generation Config
Control creativity and output behavior with generation parameters.

In [ ]:
prompt = "Generate a creative startup idea in one sentence."

print("Low Temperature (0.2) - More focused:")
response_low = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        temperature=0.2,
        max_output_tokens=100,
    ),
)
print(f"  {response_low.text}\n")

print("High Temperature (1.0) - More creative:")
response_high = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=types.GenerateContentConfig(
        temperature=1.0,
        max_output_tokens=100,
    ),
)
print(f"  {response_high.text}")

### 12. Token Counting
Count tokens for input content to manage API costs.

In [ ]:
long_text = "Artificial intelligence is transforming every industry. " * 50

token_count = client.models.count_tokens(
    model=MODEL_ID,
    contents=long_text
)

print(f"Text length: {len(long_text)} characters")
print(f"Token count: {token_count.total_tokens} tokens")

### 13. Safety Settings
Configure content safety filters for your application needs.

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Write a story about a detective solving a mystery.",
    config=types.GenerateContentConfig(
        safety_settings=[
            types.SafetySetting(
                category="HARM_CATEGORY_DANGEROUS_CONTENT",
                threshold="BLOCK_ONLY_HIGH"
            ),
        ],
    ),
)

print(response.text[:500] + "...")

---

## Summary

This notebook covered the key capabilities of **Gemini 3 Flash**:

| Feature | Use Case |
|---------|----------|
| Basic Generation | Simple text completions |
| Streaming | Real-time chat applications |
| System Instructions | Customizing model behavior |
| Multimodal | Image understanding & analysis |
| Structured Outputs | Type-safe JSON responses |
| Function Calling | Tool use & automation |
| Google Search | Real-time information grounding |
| Multi-turn Chat | Contextual conversations |
| Generation Config | Temperature, tokens control |

**Next Steps:**
- Explore the [Google AI Studio](https://aistudio.google.com/) for interactive testing
- Check out [Gemini API Documentation](https://ai.google.dev/docs) for more features
- Join the [Gen AI Experiments](https://github.com/buildfastwithai/gen-ai-experiments) community